In [ ]:
# Pacotes
import pandas as pd
import os
import shutil

In [ ]:
def filtrar_linhas(obj, string, uf, mes, ano, var):
  obj_fin = obj[obj['var1'].str.contains(string, na = False)].dropna(axis = 'columns')

  obj_fin['uf'] = uf
  obj_fin['mes'] = mes
  obj_fin['ano'] = ano
  obj_fin['var'] = var
  return obj_fin

In [ ]:
arqs = os.listdir('arqs_sisdepen')

arqs_usar = []

for arq in arqs:
  if arq in ('prjun2020.xls', 'rnjun2020.xls'):
    arqs_usar.append(arq)
    #print(arq)
  else: 
    ano = int(float(arq[7:11]))
    if ano >= 2014:
      arqs_usar.append(arq)

len(arqs_usar)

350

In [ ]:
d = {'level': ['999'], 'value': [999], 'uf': ['999'], 'mes': ['999'], 'ano': [999], 'var': ['999']}
df_infogeral = pd.DataFrame(data = d)

e = {'level': ['999'], 'total': [999], 'porc': ['999'], 'uf': ['999'], 'mes': ['999'], 'ano': [999], 'var': ['999']}
df_sala = pd.DataFrame(data = e)

aux = {'level': ['999'], 'masc': ['999'], 'fem': ['999'], 'total': [999], 'uf': ['999'], 'mes': ['999'], 'ano': [999], 'var': ['999']}
df_cons = pd.DataFrame(data = aux)

aux2 = {'level': ['999'], 'masc': ['999'], 'fem': ['999'], 'misto': ['999'], 'total': [999], 'uf': ['999'], 'mes': ['999'], 'ano': [999], 'var': ['999']}
df_totestab = pd.DataFrame(data = aux2)

df_psico = pd.DataFrame(data = d)

df_suic = pd.DataFrame(data = aux)

df_suic

,level,masc,fem,total,uf,mes,ano,var
0,999,999,999,999,999,999,999,999


In [ ]:
for arq in arqs_usar:
  # Criando variaveis a partir de slicing do nome do arq
  if arq in ('prjun2020.xls', 'rnjun2020.xls'):
    UF = arq[0:2]
    MES = arq[2:5]
    ANO = arq[5:9]
  else:
    UF = arq[0:2]
    MES = arq[3:6]
    ANO = arq[7:11]


  # Lendo os dados
  popcarcer = pd.read_excel('arqs_sisdepen' + '/' + arq,   
                            skiprows = 7, 
                            names = ["var0", "var1", "var2", "var3", "var4", "var5",
                              "var6", "var7", "var8", "var9", "var10", "var11",
                              "var12", "var13", "var14", "var15", "var16", "var17",
                              "var18", "var19", "var20", "var21", "var22"],
                            index_col = 0,
                            header = None)
  
  # Tentar: df['Meta'] = df['Meta'].fillna(method='pad') 
  
  # Drop the rows where all elements are missing.
  popcarcer = popcarcer.dropna(how = 'all')
  
  # Info
  infogeral = filtrar_linhas(popcarcer, 'População carcerária', UF, MES, ANO, 'pop')
  infogeral = infogeral.rename(columns = {list(infogeral)[0]:'level', list(infogeral)[1]:'value'})
  df_infogeral = df_infogeral.append([infogeral])

  # Total de estab
  totestab = filtrar_linhas(popcarcer, 'Total de estabelecimentos penais', UF, MES, ANO, 'pop')
  totestab = totestab.rename(columns = {list(totestab)[0]:'level', list(totestab)[1]:'masc', list(totestab)[2]:'fem', list(totestab)[3]:'misto', list(totestab)[4]:'total'})
  df_totestab = df_totestab.append([totestab])

  # Sala de Atendimento Psicologico
  sala = filtrar_linhas(popcarcer, 'Leitos Psiquiátricos|sala exclusiva de atendimento psicológico|sala de atendimento psicológico compartilhada', UF, MES, ANO, 'sala')
  sala = sala.rename(columns = {list(sala)[0]:'level', list(sala)[1]:'total', list(sala)[2]:'porc'}) 
  df_sala = df_sala.append([sala])

  # Consultas
  cons = filtrar_linhas(popcarcer, 'Consultas psicológicas', UF, MES, ANO, 'cons')
  cons = cons.rename(columns = {list(cons)[0]:'level', list(cons)[1]:'masc', list(cons)[2]:'fem', list(cons)[3]:'total'}) 
  df_cons = df_cons.append([cons])

  #print(arq)

  # Suicidios
  suic = filtrar_linhas(popcarcer, 'Óbitos', UF, MES, ANO, 'suic')
  suic = suic.rename(columns = {list(suic)[0]:'level', list(suic)[1]:'masc', list(suic)[2]:'fem', list(suic)[3]:'total'}) 
  df_suic = df_suic.append([suic])  

  # Psicologos
  psico = filtrar_linhas(popcarcer, 'Terapeutas|Psicólogos|Psiquiatras|psiquiatras', UF, MES, ANO, 'psico')
  psico = psico.rename(columns = {list(psico)[0]:'level', list(psico)[1]:'value'})
  df_psico = df_psico.append([psico]) 


In [ ]:
def ajeitar_ordenar_linhas(obj):
  obj_fin = obj.query(' uf != "999" ')
  obj_fin = obj_fin.dropna(axis = 'columns')
  obj_fin = obj_fin.sort_values(by = ['uf', 'ano', 'mes'])
  return obj_fin

In [ ]:
df_infogeral = ajeitar_ordenar_linhas(df_infogeral)
df_totestab = ajeitar_ordenar_linhas(df_totestab)
df_sala = ajeitar_ordenar_linhas(df_sala)
df_cons = ajeitar_ordenar_linhas(df_cons)
df_suic = ajeitar_ordenar_linhas(df_suic)
df_psico = ajeitar_ordenar_linhas(df_psico)

In [ ]:
df_suic

In [ ]:
# Gerando planilhas desses arqs
os.mkdir('dados')

df_infogeral.to_csv('dados/df_infogeral.csv', index = False)
df_totestab.to_csv('dados/df_totestab.csv', index = False)
df_sala.to_csv('dados/df_sala.csv', index = False)
df_cons.to_csv('dados/df_cons.csv', index = False)
df_suic.to_csv('dados/df_suic.csv', index = False)
df_psico.to_csv('dados/df_psico.csv', index = False)

In [ ]:
# Comprimindo os arqs p fazer download
shutil.make_archive('dados', 'zip', 'dados')

'/content/dados.zip'